In [1]:
from typing import Callable
from Crypto.Cipher import AES

In [2]:
def davies_meyer(
    encrypt : Callable[[bytearray , bytearray], bytearray] ,
    l_key: int ,
    l_message: int
) -> Callable[[bytearray], bytearray]:
    """
    Arguments :
        encrypt : an encryption function
        l_key : length in bytes of the keys for encrypt
        l_message : length in bytes of the messages for encrypt
    Returns :
        A compression function from messages of length l_key + l_message to
        messages of length l_message , defined by using the Davies - Meyer
        construction
    """
    def compress(message: bytearray) -> bytearray:
        """
        Arguments :
            message : a message of length l_key + l_message
        Returns :
            A message of length l_message
        """
        # Get the key by removing the last l_message bytes
        key = message[:l_key]
        # Get the message by removing the first l_key bytes
        message = message[l_key:]
        # Encrypt the message with the key XOR the key
        return bytearray(a ^ b for (a, b) in zip(encrypt(key, message), message))
    return compress


In [3]:
def pad(message: bytearray, l_block: int) -> bytearray:
    """
    Arguments :
        message : message to be padded
        l_block : length in bytes of the block
    Returns :
        extension of message that includes the length of message
        (in bytes ) in its last block
    """
    mod = len(message) % l_block
    if mod == 0:
        return message + len(message).to_bytes(l_block, "big")
    else:
        return message + (1 * (256 ** (l_block - mod - 1))).to_bytes(l_block - mod, "big") + len(message).to_bytes(l_block, "big")
    return message


In [4]:
def merkle_damgard(
    IV: bytearray, comp: Callable[[bytearray], bytearray], l_block: int
) -> Callable[[bytearray], bytearray]:
    """
    Arguments :
        IV: initialization vector for a hash function
        comp : compression function to be used in the Merkle - Damgard
        construction
        l_block : length in bytes of the blocks to be used in the Merkle - Damgard
        construction
    Returns :
        A hash function for messages of arbitrary length , defined by using
        the Merkle - Damgard construction
    """
    def hash(message: bytearray) -> bytearray:
        """
        Arguments :
            message : a message of arbitrary length
        Returns :
            A hash of the message
        """
        message = pad(message, l_block)
        h = IV
        for i in range(0, len(message), l_block):
            h = comp(message[i : i + l_block] + h)
        return h
    return hash

In [5]:
def _AES_128(key: bytearray, message: bytearray) -> bytearray:
    a = AES.new(key, AES.MODE_ECB)
    return bytearray(a.encrypt(message))

In [ ]:
if __name__ == "__main__":
    compresion = davies_meyer(_AES_128, 16, 16)
    hash = merkle_damgard(bytearray(b'1234567890123456'), compresion, 16)

    s1 = bytearray(b'Este es un mensaje de prueba para la tarea 2')
    s2 = bytearray(b'Este es un mensaje de Prueba para la tarea 2')
    s3 = bytearray(b'Un mensaje corto')
    s4 = bytearray(b'')

    h1 = hash(s1)
    h2 = hash(s2)
    h3 = hash(s3)
    h4 = hash(s4)

    assert h1 == bytearray(b'\xe9\xe8\xac\x12\\\xf2\xc8\x16\xceOV\xc5Y.T\xea')
    assert h2 == bytearray(b'\xb6\xfb\xc6a\x12\xae\x95\x1f\xda\xc5\x13\xde\x06|Q\x96')
    assert h3 == bytearray(b'\xc5\xec\xcdd\xa4(R*\xf0L*QtL\xda\x81')
    assert h4 == bytearray(b'p\xca \xd8\x9c\xeb\xe6\xb1\xce\xcf\x03\xb2\x9e\x93\x19\xbc')